In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pdb
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import numpy as np
from pathlib import Path
from glob import glob
from matplotlib.patches import Rectangle
import png
from tqdm import tqdm_notebook as tqdm
import os

In [2]:
dp = Path('../datasets/RSNA_PDC/')

DICOMS = dp/'stage_2_test_images'

PNGS = dp/'test2_png'
PNGS.mkdir(exist_ok=True)

In [3]:
def from_dicom_to_png(dicom_path, png_path):
    ds = pydicom.dcmread(dicom_path)
    shape = ds.pixel_array.shape
    # Convert to float to avoid overflow or underflow losses.
    image_2d = ds.pixel_array.astype(float)
    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0
    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    # Write the PNG file
    with open(png_path, 'wb') as png_file:
        w = png.Writer(shape[1], shape[0], greyscale=True)
        w.write(png_file, image_2d_scaled)

In [4]:
dcms = list(DICOMS.iterdir())
dcms[0:5]

[PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/2b554342-6224-4d4c-822c-48a56a522c3e.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/1a941593-8eb0-41dd-9763-da1a1c874196.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/140ebc5d-5a15-4948-b698-0a4b32d12a93.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/245beb7f-d20b-49eb-8403-09360ec9f163.dcm'),
 PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/26b4e5ff-cfa6-4b2a-b0cb-a431dd02ddd9.dcm')]

In [5]:
len(dcms)

3000

In [6]:
p = dcms[0]
p

PosixPath('../../datasets/RSNA_PDC/stage_2_test_images/2b554342-6224-4d4c-822c-48a56a522c3e.dcm')

In [7]:
b = os.path.basename(p)

In [8]:
os.path.splitext(b)[0]

'2b554342-6224-4d4c-822c-48a56a522c3e'

In [9]:
for n, i in tqdm(enumerate(dcms), total=len(dcms)):
    bn = os.path.basename(i)
    n = os.path.splitext(bn)[0]
    png_n = n+'.png'
    png_path = PNGS/png_n
    from_dicom_to_png(str(i), png_path)